In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'march'
new_month = 'april'

In [4]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2020.xlsx',\
                         sheetname='assets',index_col='Ticker')

In [5]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [6]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [7]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [8]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [9]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [10]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

In [11]:
metrics_log['elf']

Y2050 Supply         5.444802e+08
Supply % Issued      1.000000e+00
Price                5.557816e-02
Marketcap            3.026121e+07
Current Marketcap    3.026121e+07
Est Volume           4.050003e+09
dtype: float64

In [12]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [13]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [14]:
# t = 'VET'
# metrics = get_messari_data(t)
# metrics_log[t] = metrics
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# #df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#     df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#     pass
# try:
#     df.loc[t,('Category2')] = metrics['Category2']
# except:
#     pass
# try:
#     df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [15]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [16]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [17]:
#read in other data sources
onchainfx_db = df

In [18]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

145
145
145
145


In [19]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOT' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [20]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [21]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [22]:
onchainfx_db.loc['GNO']

Coin                      Gnosis
Class                          3
Sector                       NaN
Quality Exchange?              1
ERC20                          1
ICO ok?                        1
Eligible                       1
Y2050 Supply         1.10459e+06
Available Supply     1.10459e+06
Supply % Issued                1
Price                    10.2886
Marketcap            1.13647e+07
Current Marketcap    1.13647e+07
Est Volume           7.26224e+06
Name: GNO, dtype: object

In [23]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [24]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [25]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [26]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [27]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

Marketcap  Current Marketcap  Float Percent         Float  \
CENNZ  4.717700e+07       4.717700e+07            1.0  8.425212e+08   
CTXC   1.279701e+07       1.279701e+07            1.0  2.539879e+08   
DENT   1.104400e+07       1.104400e+07            1.0  1.000000e+11   
DRGN   4.559343e+06       4.559343e+06            1.0  2.384219e+08   
ETN    2.185015e+07       2.185015e+07            1.0  1.004219e+10   
GRIN   4.617916e+08       4.617916e+08            1.0  9.771011e+08   
KCS    8.536220e+07       8.536220e+07            1.0  8.185045e+07   
MKR    2.959164e+08       2.959164e+08            1.0  9.966470e+05   
R      5.116063e+06       5.116063e+06            1.0  4.844500e+08   
SMART  3.887346e+06       3.887346e+06            1.0  1.413859e+09   
SPANK  4.110594e+05       4.110594e+05            1.0  3.000434e+08   
HT     7.584842e+08       7.584842e+08            1.0  2.262948e+08   
NEXO   1.070250e+08       1.070250e+08            1.0  1.000000e+09   
ELA    2.021503e+07       2.021503e+07            1.0  1.822039e+07   
PAI    2.321569e+07       2.321569e+07            1.0  2.097694e+09   

            Price  Total Supply        Volume  Float Pass  Volume Ratio  \
CENNZ    0.055995  8.425212e+08  2.142015e+07           1      0.454038   
CTXC     0.050384  2.539879e+08  5.759476e+08           1     45.006416   
DENT     0.000110  1.000000e+11  5.097601e+07           1      4.615719   
DRGN     0.019123  2.384219e+08  6.545660e+06           1      1.435659   
ETN      0.002176  1.004219e+10  9.595960e+06           1      0.439171   
GRIN     0.472614  9.771011e+08  6.768905e+09           1     14.657922   
KCS      1.042904  8.185045e+07  1.025146e+09           1     12.009368   
MKR    296.911906  9.966470e+05  7.016121e+08           1      2.370981   
R        0.010561  4.844500e+08  1.317707e+08           1     25.756260   
SMART    0.002749  1.413859e+09  3.611728e+07           1      9.290986   
SPANK    0.001370  3.000434e+08  3.187382e+05           1      0.775407   
HT       3.351753  2.262948e+08  2.258260e+10           1     29.773332   
NEXO     0.107025  1.000000e+09  1.794920e+09           1     16.771039   
ELA      1.109473  1.822039e+07  3.755168e+08           1     18.576116   
PAI      0.011067  2.097694e+09  1.069562e+08           1      4.607065   

       Volume Pass  Other Pass  Class  ERC20  Score  Eligible  
CENNZ            1         0.0    3.0    0.0    2.0         0  
CTXC             1         0.0    2.0    0.0    2.0         0  
DENT             1         0.0    3.0    0.0    2.0         0  
DRGN             1         0.0    3.0    0.0    2.0         0  
ETN              1         0.0    3.0    0.0    2.0         0  
GRIN             1         0.0    1.0    0.0    2.0         0  
KCS              1         0.0    2.0    0.0    2.0         0  
MKR              1         0.0    3.0    0.0    2.0         0  
R                1         0.0    3.0    0.0    2.0         0  
SMART            1         0.0    3.0    0.0    2.0         0  
SPANK            1         0.0    3.0    1.0    2.0         0  
HT               1         0.0    3.0    1.0    2.0         0  
NEXO             1         0.0    3.0    1.0    2.0         0  
ELA              1         0.0    2.0    0.0    2.0         0  
PAI              1         0.0    3.0    0.0    2.0         0

In [28]:
#eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
eligible_db = eligible_db.drop('ATOM')

In [29]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

In [30]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

Marketcap  Current Marketcap  Float Percent         Float  \
BTC    1.386670e+11       1.386670e+11            1.0  2.098634e+07   
ETH    1.819658e+10       1.819658e+10            1.0  1.351356e+08   
XRP    1.612420e+10       1.612420e+10            1.0  1.000000e+11   
BCH    4.645144e+09       4.645144e+09            1.0  2.098243e+07   
BSV    3.557109e+09       3.557109e+09            1.0  2.098214e+07   
XTZ    3.307877e+09       3.307877e+09            1.0  1.923801e+09   
LTC    3.269781e+09       3.269781e+09            1.0  8.388054e+07   
EOS    3.126260e+09       3.126260e+09            1.0  1.366412e+09   
LINK   2.227100e+09       2.227100e+09            1.0  1.000000e+09   
XLM    2.014520e+09       2.014520e+09            1.0  5.000000e+10   
ADA    1.235225e+09       1.235225e+09            1.0  4.201402e+10   
BNB    1.228782e+09       1.228782e+09            1.0  1.000000e+08   
TRX    1.188482e+09       1.188482e+09            1.0  1.041451e+11   
DASH   1.181796e+09       1.181796e+09            1.0  1.760081e+07   
XMR    1.131240e+09       1.131240e+09            1.0  2.246608e+07   
ETC    9.693586e+08       9.693586e+08            1.0  1.945714e+08   
ICX    8.286757e+08       8.286757e+08            1.0  4.105533e+09   
NEO    6.787504e+08       6.787504e+08            1.0  1.000000e+08   
ZEC    6.589935e+08       6.589935e+08            1.0  2.092423e+07   
DOGE   5.120877e+08       5.120877e+08            1.0  2.806667e+11   
MIOTA  3.922394e+08       3.922394e+08            1.0  2.779530e+09   
ONT    3.662199e+08       3.662199e+08            1.0  1.000000e+09   
XEM    3.491824e+08       3.491824e+08            1.0  9.000000e+09   
RVN    3.207887e+08       3.207887e+08            1.0  2.075496e+10   
VET    2.653218e+08       2.653218e+08            1.0  8.671626e+10   
DCR    2.376243e+08       2.376243e+08            1.0  2.061974e+07   
LSK    2.293425e+08       2.293425e+08            1.0  2.315385e+08   
BAT    2.217243e+08       2.217243e+08            1.0  1.500000e+09   
BTG    1.565422e+08       1.565422e+08            1.0  2.098311e+07   
ZRX    1.535070e+08       1.535070e+08            1.0  1.000000e+09   
...             ...                ...            ...           ...   
WTC    1.517802e+07       1.517802e+07            1.0  6.965028e+07   
LOOM   1.395665e+07       1.395665e+07            1.0  9.695203e+08   
NAS    1.259061e+07       1.259061e+07            1.0  5.195259e+07   
MET    1.220991e+07       1.220991e+07            1.0  4.316032e+07   
BNT    1.196307e+07       1.196307e+07            1.0  6.914864e+07   
GBYTE  1.137471e+07       1.137471e+07            1.0  7.417287e+05   
IOTX   1.136762e+07       1.136762e+07            1.0  5.400000e+09   
GNO    1.136474e+07       1.136474e+07            1.0  1.104590e+06   
VTC    1.117585e+07       1.117585e+07            1.0  5.403385e+07   
PPT    1.096210e+07       1.096210e+07            1.0  5.325225e+07   
SYS    9.986887e+06       9.986887e+06            1.0  5.815755e+08   
LBC    9.300113e+06       9.300113e+06            1.0  1.083202e+09   
NXT    8.514592e+06       8.514592e+06            1.0  9.989999e+08   
EMC2   8.204710e+06       8.204710e+06            1.0  2.195781e+08   
STORM  8.078996e+06       8.078996e+06            1.0  7.615901e+09   
ENG    7.518784e+06       7.518784e+06            1.0  7.483617e+07   
EDO    7.175746e+06       7.175746e+06            1.0  7.191426e+07   
CND    6.225303e+06       6.225303e+06            1.0  1.923289e+09   
MFT    5.953028e+06       5.953028e+06            1.0  1.000000e+10   
ETHOS  5.590447e+06       5.590447e+06            1.0  2.222952e+08   
PAY    5.514588e+06       5.514588e+06            1.0  2.052183e+08   
BLOCK  5.361945e+06       5.361945e+06            1.0  6.688239e+06   
QKC    5.302346e+06       5.302346e+06            1.0  2.360183e+09   
RDN    3.706469e+06       3.706469e+06            1.0  5.079394e+07   
DNT    3.703865e+06       3.703865

In [31]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained XLM
Retained ADA
10 full! Do not add  LINK
10 full! Do not add  BNB


In [32]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained LSK
Retained BAT
Retained ZRX
Retained QTUM
20 full! Do not add  BTG
20 full! Do not add  NMR
20 full! Do not add  MONA
20 full! Do not add  SC


In [33]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained GXC
Retained STORJ
Retained NULS
Retained GNT
Retained ARDR
Retained ELF
Retained MAID
Retained WAN
Retained FUN
Retained CVC
40 full! Do not add  WAX
40 full! Do not add  QASH
40 full! Do not add  STRAT
40 full! Do not add  LRC
40 full! Do not add  MANA
40 full! Do not add  ANT
40 full! Do not add  POWR
40 full! Do not add  MTL
40 full! Do not add  RDD
40 full! Do not add  POLY


In [34]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [35]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [36]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

10 adds []
10 deletes []
10 count members: 10

40 adds []
40 deletes []
40 count members: 40

Currency adds []
Currency deletes []
Currency count members: 19

Total adds []
Total deletes []
Total count members: 70

Platform adds []
Platform deletes []
Platform count members: 26

ETH adds []
ETH deletes []
ETH count members: 20

Application adds []
Application deletes []
Application count members: 25

20 adds []
20 deletes []
20 count members: 20

Sector count members: 70


In [37]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])